In [1]:
import os
os.chdir("..")
assert 'PromptDataExtract' in os.getcwd()

In [39]:
import json
from backend.utils import Frame
from pymongo import MongoClient
from backend import sett

In [41]:
prop_meta = json.load(open('data/property_metadata.json'))

In [34]:
def connect():
    client = MongoClient(
        host='localhost',
        port=8161,
    )
    db = client['corpus']
    return db

mongo = connect()
collection = mongo['modular_run_4_PS']

In [16]:
df = Frame()

In [44]:
egnames = prop_meta['bandgap']['property_list']
tgnames = prop_meta['glass transition temperature']['property_list']

In [47]:
records = list(collection.find(
    {'$or': [
        {"material_records.property_record.entity_name": {'$in': egnames }},
        {"material_records.property_record.entity_name": {'$in': tgnames }},
    ]}
))

for record in records:
    for mat in record['material_records']:
        prop = mat['property_record']
        if prop['entity_name'] in egnames:
            # bandgap
            df.add(doi=record['DOI'], prop='bandgap', value=prop['property_numeric_value'])
        elif prop['entity_name'] in tgnames:
            # tg
            df.add(doi=record['DOI'], prop='tg', value=prop['property_numeric_value'])


df.save('notebooks/Abstract-Extracted_Data.csv')
